In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from chart_studio.plotly import plotly as py
import plotly.graph_objects as go

%matplotlib inline

import requests

In [2]:
df = pd.read_csv('data_cleaned.csv')

In [3]:
df.head(3)

,Address,Prisantydning,Fellesgjeld,Omkostninger,Totalpris,Felleskost,Boligtype,Eieform_bolig,Soverom,Primærrom,Bruksareal,Etasje,Byggeår,Energimerking,Beskrivelse,Eiendomsmegler
0,"St. Halvards gate 25E, 0192 Oslo",3850000.0,0.0,76000.0,3926000.0,3964.0,Leilighet,Eier (Selveier),1,48 m²,48 m²,1940,E - gul,NaN,Lys og lekker leilighet med solrik balkong. Go...,Aktiv Grønland/Bjørvika
1,"Selvbyggerveien 143, 0591 Oslo",3200000.0,73058.0,8009.0,3281067.0,3281067.0,Leilighet,Andel,1,35 m²,37 m²,1,1953,G - rød,Romslig og attraktiv 2-roms | Hyggelig utsikt ...,Schala & Partners Årvoll
2,"Gregers Grams vei 6C, 0382 Oslo",18250000.0,0.0,471920.0,18721920.0,2167.0,Enebolig,Eier (Selveier),4,208 m²,242 m²,2016,B - gul,NaN,Tiltalende og moderne kjedet enebolig m/ takte...,PrivatMegleren Dyve & Partnere


In [4]:
df.shape

(926, 16)

In [5]:
df.isna().sum()

Address             0
Prisantydning       0
Fellesgjeld         0
Omkostninger        0
Totalpris           0
Felleskost          0
Boligtype           0
Eieform_bolig       0
Soverom             0
Primærrom           0
Bruksareal          0
Etasje             11
Byggeår             0
Energimerking     235
Beskrivelse         0
Eiendomsmegler    198
dtype: int64

In [6]:
df.columns

Index(['Address', 'Prisantydning', 'Fellesgjeld', 'Omkostninger', 'Totalpris',
       'Felleskost', 'Boligtype', 'Eieform_bolig', 'Soverom', 'Primærrom',
       'Bruksareal', 'Etasje', 'Byggeår', 'Energimerking', 'Beskrivelse',
       'Eiendomsmegler'],
      dtype='object')

### Check these columns ['Boligtype',  'Eieform_bolig',  'Soverom',  'Primærrom', 'Bruksareal', 'Etasje', 'Byggeår', 'Energimerking',  'Beskrivelse', 'Eiendomsmegler' ]. column by column

#### Boligtype

In [7]:
# no missing values, no corrupt data. Nothing to do with this column for now. 

df['Boligtype'].unique()

array(['Leilighet', 'Enebolig', 'Tomannsbolig', 'Rekkehus'], dtype=object)

#### Eieform_bolig

In [8]:
df['Eieform_bolig'].unique()

array(['Eier (Selveier)', 'Andel', 'Aksje'], dtype=object)

In [9]:
# here I will replace 'Eier (Selveier)' with only 'Selveier'

df['Eieform_bolig'].replace('Eier (Selveier)', 'Selveier', inplace = True)

In [10]:
df['Eieform_bolig'].unique()

array(['Selveier', 'Andel', 'Aksje'], dtype=object)

#### Soverom

In [11]:
# Some of the values (containing m²) in this column, belongs to either the Primærrom' or to the 'Bruksareal' column.

df['Soverom'].unique()

array(['1', '4', '2', '5', '3', '0', '8', '6', '143 m²', '30 m²', '35 m²',
       '23 m²', '208 m²', '7', '105 m²', '65 m²', '24 m²', '18 m²',
       '232 m²', '25 m²', '308 m²', '31 m²', '54 m²', '42 m²', '32 m²',
       '452 m²', '130 m²', '28 m²', '34 m²', '62 m²', '9'], dtype=object)

In [12]:
# get these rows and investigate.

filt = (df['Soverom'].str.contains('m²'))
df.loc[filt, ['Soverom', 'Primærrom', 'Bruksareal']]

,Soverom,Primærrom,Bruksareal
106,143 m²,152 m²,1982
141,30 m²,38 m²,2
165,35 m²,45 m²,5
191,23 m²,23 m²,1959
202,208 m²,208 m²,3
275,105 m²,105 m²,2
282,65 m²,65 m²,2
339,24 m²,24 m²,5
369,18 m²,20 m²,1898
438,232 m²,320 m²,1996


In [13]:
# the values of Soverom column in these rows belongs to Bruksareal. SWAP

df.loc[filt, ['Bruksareal', 'Soverom']] = df.loc[filt, ['Soverom', 'Bruksareal']].values

In [14]:
df.loc[filt]

,Address,Prisantydning,Fellesgjeld,Omkostninger,Totalpris,Felleskost,Boligtype,Eieform_bolig,Soverom,Primærrom,Bruksareal,Etasje,Byggeår,Energimerking,Beskrivelse,Eiendomsmegler
106,"Nordåssløyfa 25B, 1251 Oslo",4800000.0,169000.0,15554.0,4984554.0,4984554.00,Rekkehus,Andel,1982,152 m²,143 m²,E - lysegrønn,87078 m² (eiet),NaN,Pent rekkehus med 4 soverom og 2 bad. Gode ute...,NaN
141,"Bjerregaards gate 19, 0172 Oslo",3250000.0,368000.0,5432.0,3623432.0,3623432.00,Leilighet,Andel,2,38 m²,30 m²,1960,D - mørkegrønn,NaN,Rålekker leilighet med nytt bad og balkong mid...,Proaktiv Eiendomsmegling
165,"Rødtvetveien 16, 0955 Oslo",2750000.0,42442.0,8060.0,2800502.0,2800502.00,Leilighet,Andel,5,45 m²,35 m²,1963,1,NaN,Arealeffektiv 1-roms med alkove - Innglasset b...,NaN
191,"Prost Hallings vei 39 A, 0666 Oslo",2610000.0,40000.0,23145.0,2673145.0,2673145.00,Leilighet,Aksje,1959,23 m²,23 m²,G - rød,1,NaN,Innbydende endeleilighet i høy 1. etasje. Omfa...,PrivatMegleren Park
202,"Lambrechts gate 8, 0262 Oslo",19900000.0,333517.0,517362.0,20750879.0,20750879.00,Leilighet,Selveier,3,208 m²,208 m²,1890,1054 m² (eiet),NaN,"Nyoppusset i 21! Eksklusiv leilighet i flott, ...",Aktiv Grünerløkka
275,"Sloreåsen 8, 1257 Oslo",5300000.0,104126.0,15554.0,5419680.0,5419680.00,Leilighet,Andel,2,105 m²,105 m²,1980,E - lysegrønn,NaN,Flott og påkostet 3(4)-roms | vestvendt terras...,PrivatMegleren Panorama
282,"Nedre Prinsdals vei 59P, 1263 Oslo",2750000.0,147000.0,8060.0,2905060.0,2905060.00,Leilighet,Andel,2,65 m²,65 m²,1986,G - rød,NaN,Prinsdal- Pen seniorbolig (55+) med solrik bal...,NaN
339,"Lambertseterveien 35, 1150 Oslo",2690000.0,199717.0,8070.0,2897787.0,2897787.00,Leilighet,Andel,5,24 m²,24 m²,1961,D - rød,NaN,Nyoppusset leilighet |Alkove m/vindu | Nytt kj...,PrivatMegleren Solli & Partnere
369,"Holbergs gate 11E, 0166 Oslo",2690000.0,67763.0,80462.0,2838225.0,2838225.00,Leilighet,Selveier,1898,20 m²,18 m²,434 m² (eiet),23 m²,NaN,Super 1-roms investeringsobjekt med uteplass. ...,Sem & Johnsen
438,"Doktor Holms vei 4G, 0787 Oslo",19800000.0,0.0,510670.0,20310670.0,1256.67,Enebolig,Selveier,1996,320 m²,232 m²,E - rød,769 m² (eiet),NaN,"Enebolig over to plan med flott utsikt, solfyl...",Privatmegleren Tjuvholmen


In [15]:
# after swaping, I see that some values in the Soverom column belongs to the Byggeår column.
# get these rows where the number of Soverom (bedrooms) are higher than 20.

filt = (df['Soverom'].astype(int) > 20)
df.loc[filt]

,Address,Prisantydning,Fellesgjeld,Omkostninger,Totalpris,Felleskost,Boligtype,Eieform_bolig,Soverom,Primærrom,Bruksareal,Etasje,Byggeår,Energimerking,Beskrivelse,Eiendomsmegler
106,"Nordåssløyfa 25B, 1251 Oslo",4800000.0,169000.0,15554.0,4984554.0,4984554.00,Rekkehus,Andel,1982,152 m²,143 m²,E - lysegrønn,87078 m² (eiet),NaN,Pent rekkehus med 4 soverom og 2 bad. Gode ute...,NaN
191,"Prost Hallings vei 39 A, 0666 Oslo",2610000.0,40000.0,23145.0,2673145.0,2673145.00,Leilighet,Aksje,1959,23 m²,23 m²,G - rød,1,NaN,Innbydende endeleilighet i høy 1. etasje. Omfa...,PrivatMegleren Park
369,"Holbergs gate 11E, 0166 Oslo",2690000.0,67763.0,80462.0,2838225.0,2838225.00,Leilighet,Selveier,1898,20 m²,18 m²,434 m² (eiet),23 m²,NaN,Super 1-roms investeringsobjekt med uteplass. ...,Sem & Johnsen
438,"Doktor Holms vei 4G, 0787 Oslo",19800000.0,0.0,510670.0,20310670.0,1256.67,Enebolig,Selveier,1996,320 m²,232 m²,E - rød,769 m² (eiet),NaN,"Enebolig over to plan med flott utsikt, solfyl...",Privatmegleren Tjuvholmen
476,"Jomfrubråtveien 85 og 85B, 1179 Oslo",32500000.0,0.0,1040670.0,33540670.0,0.00,Enebolig,Selveier,1923,507 m²,308 m²,D - mørkegrønn,1379 m² (eiet),NaN,To eneboliger med meget attraktiv og populær b...,PrivatMegleren Panorama
626,"Motzfeldts gate 30, 0561 Oslo",35000000.0,0.0,0.0,35875000.0,5754.83,Leilighet,Selveier,1892,717 m²,452 m²,295 m² (eiet),798 m²,NaN,Tøyen: Hel bygård med 8 leiligheter med mange ...,Aktiv Torshov
663,"Skausnaret 12, 1262 Oslo",11000000.0,0.0,290670.0,11290670.0,0.00,Tomannsbolig,Selveier,1952,165 m²,130 m²,2766 m² (eiet),187 m²,NaN,Eldre tomannsbolig med tomt på hele 2.766 m². ...,PrivatMegleren Panorama
775,"Holmenkollveien 68, 0784 Oslo",34000000.0,0.0,865670.0,34865670.0,2526.50,Tomannsbolig,Selveier,1959,456 m²,452 m²,G - rød,2170 m² (eiet),NaN,Stor villa-eiendom over 2 plan m/uhindret pano...,PrivatMegleren Dyve & Partnere


In [16]:
# seems like these rows have corrupt data in 'Soverom', 'Etasje', 'Byggeår' and 'Energimerking'.
# ToDo:
# 1. move values that contains '-' from 'Etasje' to 'Energimerking'
# 2. move values from 'Soverom' to 'Byggeår'
# 3. check the beskrivelse column, cuz we might find info about the nuumber of Soverom (bedrooms) and the Etasje (floot)

In [17]:
df.loc[filt, 'Energimerking'] = df.loc[filt, 'Etasje'].apply(lambda x: x if '-' in x else np.NaN)

In [18]:
df.loc[filt]

,Address,Prisantydning,Fellesgjeld,Omkostninger,Totalpris,Felleskost,Boligtype,Eieform_bolig,Soverom,Primærrom,Bruksareal,Etasje,Byggeår,Energimerking,Beskrivelse,Eiendomsmegler
106,"Nordåssløyfa 25B, 1251 Oslo",4800000.0,169000.0,15554.0,4984554.0,4984554.00,Rekkehus,Andel,1982,152 m²,143 m²,E - lysegrønn,87078 m² (eiet),E - lysegrønn,Pent rekkehus med 4 soverom og 2 bad. Gode ute...,NaN
191,"Prost Hallings vei 39 A, 0666 Oslo",2610000.0,40000.0,23145.0,2673145.0,2673145.00,Leilighet,Aksje,1959,23 m²,23 m²,G - rød,1,G - rød,Innbydende endeleilighet i høy 1. etasje. Omfa...,PrivatMegleren Park
369,"Holbergs gate 11E, 0166 Oslo",2690000.0,67763.0,80462.0,2838225.0,2838225.00,Leilighet,Selveier,1898,20 m²,18 m²,434 m² (eiet),23 m²,NaN,Super 1-roms investeringsobjekt med uteplass. ...,Sem & Johnsen
438,"Doktor Holms vei 4G, 0787 Oslo",19800000.0,0.0,510670.0,20310670.0,1256.67,Enebolig,Selveier,1996,320 m²,232 m²,E - rød,769 m² (eiet),E - rød,"Enebolig over to plan med flott utsikt, solfyl...",Privatmegleren Tjuvholmen
476,"Jomfrubråtveien 85 og 85B, 1179 Oslo",32500000.0,0.0,1040670.0,33540670.0,0.00,Enebolig,Selveier,1923,507 m²,308 m²,D - mørkegrønn,1379 m² (eiet),D - mørkegrønn,To eneboliger med meget attraktiv og populær b...,PrivatMegleren Panorama
626,"Motzfeldts gate 30, 0561 Oslo",35000000.0,0.0,0.0,35875000.0,5754.83,Leilighet,Selveier,1892,717 m²,452 m²,295 m² (eiet),798 m²,NaN,Tøyen: Hel bygård med 8 leiligheter med mange ...,Aktiv Torshov
663,"Skausnaret 12, 1262 Oslo",11000000.0,0.0,290670.0,11290670.0,0.00,Tomannsbolig,Selveier,1952,165 m²,130 m²,2766 m² (eiet),187 m²,NaN,Eldre tomannsbolig med tomt på hele 2.766 m². ...,PrivatMegleren Panorama
775,"Holmenkollveien 68, 0784 Oslo",34000000.0,0.0,865670.0,34865670.0,2526.50,Tomannsbolig,Selveier,1959,456 m²,452 m²,G - rød,2170 m² (eiet),G - rød,Stor villa-eiendom over 2 plan m/uhindret pano...,PrivatMegleren Dyve & Partnere


In [19]:
# 'Byggeår' for these rows has only corrupt values except for the row '191'. where the values belongs to the Etasje column
# I will correct the value of the Etasje cell manually
# after that set values of 'Etasje' to be NaN

df.loc[191, 'Etasje'] = '1'
df.loc[filt, 'Etasje'] = np.NaN

In [20]:
# now move values from 'Soverom' to 'Byggeår'
# after that set values of 'Soverom' to be NaN

df.loc[filt, 'Byggeår'] = df.loc[filt, 'Soverom'].values
df.loc[filt, 'Soverom'] = np.NaN

In [21]:
df.loc[filt]

,Address,Prisantydning,Fellesgjeld,Omkostninger,Totalpris,Felleskost,Boligtype,Eieform_bolig,Soverom,Primærrom,Bruksareal,Etasje,Byggeår,Energimerking,Beskrivelse,Eiendomsmegler
106,"Nordåssløyfa 25B, 1251 Oslo",4800000.0,169000.0,15554.0,4984554.0,4984554.00,Rekkehus,Andel,NaN,152 m²,143 m²,NaN,1982,E - lysegrønn,Pent rekkehus med 4 soverom og 2 bad. Gode ute...,NaN
191,"Prost Hallings vei 39 A, 0666 Oslo",2610000.0,40000.0,23145.0,2673145.0,2673145.00,Leilighet,Aksje,NaN,23 m²,23 m²,NaN,1959,G - rød,Innbydende endeleilighet i høy 1. etasje. Omfa...,PrivatMegleren Park
369,"Holbergs gate 11E, 0166 Oslo",2690000.0,67763.0,80462.0,2838225.0,2838225.00,Leilighet,Selveier,NaN,20 m²,18 m²,NaN,1898,NaN,Super 1-roms investeringsobjekt med uteplass. ...,Sem & Johnsen
438,"Doktor Holms vei 4G, 0787 Oslo",19800000.0,0.0,510670.0,20310670.0,1256.67,Enebolig,Selveier,NaN,320 m²,232 m²,NaN,1996,E - rød,"Enebolig over to plan med flott utsikt, solfyl...",Privatmegleren Tjuvholmen
476,"Jomfrubråtveien 85 og 85B, 1179 Oslo",32500000.0,0.0,1040670.0,33540670.0,0.00,Enebolig,Selveier,NaN,507 m²,308 m²,NaN,1923,D - mørkegrønn,To eneboliger med meget attraktiv og populær b...,PrivatMegleren Panorama
626,"Motzfeldts gate 30, 0561 Oslo",35000000.0,0.0,0.0,35875000.0,5754.83,Leilighet,Selveier,NaN,717 m²,452 m²,NaN,1892,NaN,Tøyen: Hel bygård med 8 leiligheter med mange ...,Aktiv Torshov
663,"Skausnaret 12, 1262 Oslo",11000000.0,0.0,290670.0,11290670.0,0.00,Tomannsbolig,Selveier,NaN,165 m²,130 m²,NaN,1952,NaN,Eldre tomannsbolig med tomt på hele 2.766 m². ...,PrivatMegleren Panorama
775,"Holmenkollveien 68, 0784 Oslo",34000000.0,0.0,865670.0,34865670.0,2526.50,Tomannsbolig,Selveier,NaN,456 m²,452 m²,NaN,1959,G - rød,Stor villa-eiendom over 2 plan m/uhindret pano...,PrivatMegleren Dyve & Partnere


In [22]:
# now the Soverom and Etasje are missing.Since the number of rows are not that high, i will check if i can find any info 
# about these values in the Beskrivelse.

for idx in df.loc[filt].index:
    print(idx,':', df.loc[idx, 'Beskrivelse'])

106 : Pent rekkehus med 4 soverom og 2 bad. Gode uteområder. Sjøgløtt. Attraktiv og barnevennlig beliggenhet. Garasjeplass!
191 : Innbydende endeleilighet i høy 1. etasje. Omfattende oppusset i 2020/2021. Vestvendt, stor og overbygd balkong.
369 : Super 1-roms investeringsobjekt med uteplass. Ombygget stallbygning i indre gård. Rolig og meget sentral beliggenhet.
438 : Enebolig over to plan med flott utsikt, solfylt tomt og dobbelgarasje. Peis. Stort potensial
476 : To eneboliger med meget attraktiv og populær beliggenhet - Store muligheter - Samlet tomt på 1 379m².
626 : Tøyen: Hel bygård med 8 leiligheter med mange muligheter, stort råloft og et bruksareal på 717 kvm.
663 : Eldre tomannsbolig med tomt på hele 2.766 m². Spennende utviklingspotensial. Rammetillatelse for fem rekkehus.
775 : Stor villa-eiendom over 2 plan m/uhindret panorama og skjermet beliggenhet - Ca. 2,2 mål tomt - Svømmebasseng - Hybel


In [23]:
df.loc[106, ['Soverom', 'Etasje']] = 4, 1
df.loc[191, ['Soverom', 'Etasje']] = 0, 1 # soverom = 0 because the areal is only 23m²
df.loc[369, ['Soverom', 'Etasje']] = 1, 1
df.loc[438, ['Soverom', 'Etasje']] = 5, 1 # soverom = 5 because house with loc[106] has 4 rooms and it has less areal than this one.
df.loc[663, ['Soverom', 'Etasje']] = 8, 1
df.loc[775, ['Soverom', 'Etasje']] = 8, 1

In [24]:
# drop these rows, because it's more like projects and not houses.

df.drop([476, 626], inplace = True)

In [25]:
df.reset_index(inplace = True)

In [26]:
df.drop('index', axis = 1, inplace = True)

#### Primærrom

In [27]:
# get the rows where the 'Primærrom' doesn't contains m²
# we get only one cell and its a valid value when we compare it the value of the 'Bruksareal', 
# so I don't have to do anything for now

filt = (df['Primærrom'].str.contains('m²'))
df.loc[~filt]

,Address,Prisantydning,Fellesgjeld,Omkostninger,Totalpris,Felleskost,Boligtype,Eieform_bolig,Soverom,Primærrom,Bruksareal,Etasje,Byggeår,Energimerking,Beskrivelse,Eiendomsmegler
787,"Lyberget 7, 1295 Oslo",20000000.0,0.0,501342.0,20501342.0,672.08,Enebolig,Selveier,5,20.780,20781 m² (eiet),1,1901,NaN,Enebolig med tomt på hele 20.780 kvm. Mulighet...,Krogsveen avd. Nordstrand


#### Bruksareal

In [28]:
df['Bruksareal'].unique()

array(['48 m²', '37 m²', '242 m²', '54 m²', '72 m²', '59 m²', '85 m²',
       '58 m²', '60 m²', '46 m²', '150 m²', '84 m²', '91 m²', '66 m²',
       '17 m²', '69 m²', '42 m²', '100 m²', '145 m²', '82 m²', '71 m²',
       '99 m²', '57 m²', '103 m²', '87 m²', '15 m²', '52 m²', '74 m²',
       '70 m²', '51 m²', '67 m²', '83 m²', '30 m²', '86 m²', '246 m²',
       '231 m²', '44 m²', '53 m²', '61 m²', '39 m²', '40 m²', '104 m²',
       '115 m²', '41 m²', '62 m²', '92 m²', '34 m²', '296 m²', '75 m²',
       '45 m²', '63 m²', '175 m²', '202 m²', '49 m²', '348 m²', '140 m²',
       '174 m²', '160 m²', '79 m²', '36 m²', '78 m²', '183 m²', '136 m²',
       '76 m²', '90 m²', '93 m²', '677 m²', '55 m²', '73 m²', '97 m²',
       '143 m²', '65 m²', '605 m²', '124 m²', '29 m²', '22 m²', '220 m²',
       '110 m²', '96 m²', '56 m²', '218 m²', '18 m²', '80 m²', '88 m²',
       '253 m²', '95 m²', '250 m²', '68 m²', '47 m²', '344 m²', '458 m²',
       '157 m²', '102 m²', '35 m²', '119 m²', '101 m²', '98 m

In [29]:
df['Bruksareal'].replace('(eiet)', '', inplace = True)

In [30]:
# get the rows where the 'Bruksareal' doesn't contains m²

filt = (df['Bruksareal'].str.contains('m²'))
df.loc[~filt]

,Address,Prisantydning,Fellesgjeld,Omkostninger,Totalpris,Felleskost,Boligtype,Eieform_bolig,Soverom,Primærrom,Bruksareal,Etasje,Byggeår,Energimerking,Beskrivelse,Eiendomsmegler
198,"Tuengen allé 1A, 0374 Oslo",28000000.0,0.0,708270.0,28708270.0,6380.0,Leilighet,Selveier,3,156 m²,2,2021,A - lysegrønn,NaN,Vinderen - Ny eksklusiv toppleilighet - Privat...,Advokat Ek
320,"Grindbakken 72, 0764 Oslo",9900000.0,0.0,259042.0,10159042.0,3847.0,Leilighet,Selveier,3,123 m²,3,1984,G - oransje,NaN,"Lys, gjennomgående og oppusset 4-roms med fant...",PrivatMegleren Park
516,"Hauketoveien 15B, 1266 Oslo",6990000.0,0.0,176120.0,7166120.0,0.0,Tomannsbolig,Selveier,5,114 m²,2021,6,790 m² (eiet),NaN,Ny 1/2 part tomannsbolig med hele 5 soverom og...,EIE eiendomsmegling Bærum
518,"Aslakveien 24A, 0753 Oslo",6950000.0,0.0,183392.0,7133392.0,3367.0,Leilighet,Selveier,2,74 m²,5,2020,3,NaN,Lekker & gjennomgående 3-r selveier fra 2020 i...,Nordvik Frogner
563,"Løchenveien 14 - B1, 0286 Oslo",35000000.0,0.0,172792.0,35172792.0,0.0,Leilighet,Selveier,4,172 m²,4863,2,2021,NaN,Eksklusiv leilighet (B1) på 172 m2. Egen innga...,Sem & Johnsen


In [31]:
# we get 5 rows with corrupt values. 

df.loc[~filt, 'Energimerking'] = df.loc[~filt, 'Byggeår'].apply(lambda x: x if '-' in x else np.NAN)

In [32]:
df.loc[~filt]

,Address,Prisantydning,Fellesgjeld,Omkostninger,Totalpris,Felleskost,Boligtype,Eieform_bolig,Soverom,Primærrom,Bruksareal,Etasje,Byggeår,Energimerking,Beskrivelse,Eiendomsmegler
198,"Tuengen allé 1A, 0374 Oslo",28000000.0,0.0,708270.0,28708270.0,6380.0,Leilighet,Selveier,3,156 m²,2,2021,A - lysegrønn,A - lysegrønn,Vinderen - Ny eksklusiv toppleilighet - Privat...,Advokat Ek
320,"Grindbakken 72, 0764 Oslo",9900000.0,0.0,259042.0,10159042.0,3847.0,Leilighet,Selveier,3,123 m²,3,1984,G - oransje,G - oransje,"Lys, gjennomgående og oppusset 4-roms med fant...",PrivatMegleren Park
516,"Hauketoveien 15B, 1266 Oslo",6990000.0,0.0,176120.0,7166120.0,0.0,Tomannsbolig,Selveier,5,114 m²,2021,6,790 m² (eiet),NaN,Ny 1/2 part tomannsbolig med hele 5 soverom og...,EIE eiendomsmegling Bærum
518,"Aslakveien 24A, 0753 Oslo",6950000.0,0.0,183392.0,7133392.0,3367.0,Leilighet,Selveier,2,74 m²,5,2020,3,NaN,Lekker & gjennomgående 3-r selveier fra 2020 i...,Nordvik Frogner
563,"Løchenveien 14 - B1, 0286 Oslo",35000000.0,0.0,172792.0,35172792.0,0.0,Leilighet,Selveier,4,172 m²,4863,2,2021,NaN,Eksklusiv leilighet (B1) på 172 m2. Egen innga...,Sem & Johnsen


In [33]:
# len(int(x['Etasje'])) > 3, because Byggeår should be of a lenth of 4.

df.loc[~filt, 'Byggeår'] = df.loc[~filt].apply(lambda x: x['Etasje'] if len(x['Etasje']) > 3 else x['Byggeår'], axis = 1)

In [34]:
df.loc[~filt]

,Address,Prisantydning,Fellesgjeld,Omkostninger,Totalpris,Felleskost,Boligtype,Eieform_bolig,Soverom,Primærrom,Bruksareal,Etasje,Byggeår,Energimerking,Beskrivelse,Eiendomsmegler
198,"Tuengen allé 1A, 0374 Oslo",28000000.0,0.0,708270.0,28708270.0,6380.0,Leilighet,Selveier,3,156 m²,2,2021,2021,A - lysegrønn,Vinderen - Ny eksklusiv toppleilighet - Privat...,Advokat Ek
320,"Grindbakken 72, 0764 Oslo",9900000.0,0.0,259042.0,10159042.0,3847.0,Leilighet,Selveier,3,123 m²,3,1984,1984,G - oransje,"Lys, gjennomgående og oppusset 4-roms med fant...",PrivatMegleren Park
516,"Hauketoveien 15B, 1266 Oslo",6990000.0,0.0,176120.0,7166120.0,0.0,Tomannsbolig,Selveier,5,114 m²,2021,6,790 m² (eiet),NaN,Ny 1/2 part tomannsbolig med hele 5 soverom og...,EIE eiendomsmegling Bærum
518,"Aslakveien 24A, 0753 Oslo",6950000.0,0.0,183392.0,7133392.0,3367.0,Leilighet,Selveier,2,74 m²,5,2020,2020,NaN,Lekker & gjennomgående 3-r selveier fra 2020 i...,Nordvik Frogner
563,"Løchenveien 14 - B1, 0286 Oslo",35000000.0,0.0,172792.0,35172792.0,0.0,Leilighet,Selveier,4,172 m²,4863,2,2021,NaN,Eksklusiv leilighet (B1) på 172 m2. Egen innga...,Sem & Johnsen


In [35]:
df.loc[~filt, 'Etasje'] = df.loc[~filt].apply(lambda x: x['Bruksareal'] if int(x['Bruksareal']) < 10 else x['Etasje'], axis = 1)

In [36]:
df.loc[~filt]

,Address,Prisantydning,Fellesgjeld,Omkostninger,Totalpris,Felleskost,Boligtype,Eieform_bolig,Soverom,Primærrom,Bruksareal,Etasje,Byggeår,Energimerking,Beskrivelse,Eiendomsmegler
198,"Tuengen allé 1A, 0374 Oslo",28000000.0,0.0,708270.0,28708270.0,6380.0,Leilighet,Selveier,3,156 m²,2,2,2021,A - lysegrønn,Vinderen - Ny eksklusiv toppleilighet - Privat...,Advokat Ek
320,"Grindbakken 72, 0764 Oslo",9900000.0,0.0,259042.0,10159042.0,3847.0,Leilighet,Selveier,3,123 m²,3,3,1984,G - oransje,"Lys, gjennomgående og oppusset 4-roms med fant...",PrivatMegleren Park
516,"Hauketoveien 15B, 1266 Oslo",6990000.0,0.0,176120.0,7166120.0,0.0,Tomannsbolig,Selveier,5,114 m²,2021,6,790 m² (eiet),NaN,Ny 1/2 part tomannsbolig med hele 5 soverom og...,EIE eiendomsmegling Bærum
518,"Aslakveien 24A, 0753 Oslo",6950000.0,0.0,183392.0,7133392.0,3367.0,Leilighet,Selveier,2,74 m²,5,5,2020,NaN,Lekker & gjennomgående 3-r selveier fra 2020 i...,Nordvik Frogner
563,"Løchenveien 14 - B1, 0286 Oslo",35000000.0,0.0,172792.0,35172792.0,0.0,Leilighet,Selveier,4,172 m²,4863,2,2021,NaN,Eksklusiv leilighet (B1) på 172 m2. Egen innga...,Sem & Johnsen


In [37]:
df.loc[516, ['Bruksareal', 'Byggeår']] = df.loc[516, ['Byggeår', 'Bruksareal']].values

In [38]:
df.loc[~filt]

,Address,Prisantydning,Fellesgjeld,Omkostninger,Totalpris,Felleskost,Boligtype,Eieform_bolig,Soverom,Primærrom,Bruksareal,Etasje,Byggeår,Energimerking,Beskrivelse,Eiendomsmegler
198,"Tuengen allé 1A, 0374 Oslo",28000000.0,0.0,708270.0,28708270.0,6380.0,Leilighet,Selveier,3,156 m²,2,2,2021,A - lysegrønn,Vinderen - Ny eksklusiv toppleilighet - Privat...,Advokat Ek
320,"Grindbakken 72, 0764 Oslo",9900000.0,0.0,259042.0,10159042.0,3847.0,Leilighet,Selveier,3,123 m²,3,3,1984,G - oransje,"Lys, gjennomgående og oppusset 4-roms med fant...",PrivatMegleren Park
516,"Hauketoveien 15B, 1266 Oslo",6990000.0,0.0,176120.0,7166120.0,0.0,Tomannsbolig,Selveier,5,114 m²,790 m² (eiet),6,2021,NaN,Ny 1/2 part tomannsbolig med hele 5 soverom og...,EIE eiendomsmegling Bærum
518,"Aslakveien 24A, 0753 Oslo",6950000.0,0.0,183392.0,7133392.0,3367.0,Leilighet,Selveier,2,74 m²,5,5,2020,NaN,Lekker & gjennomgående 3-r selveier fra 2020 i...,Nordvik Frogner
563,"Løchenveien 14 - B1, 0286 Oslo",35000000.0,0.0,172792.0,35172792.0,0.0,Leilighet,Selveier,4,172 m²,4863,2,2021,NaN,Eksklusiv leilighet (B1) på 172 m2. Egen innga...,Sem & Johnsen


In [39]:
df.loc[~filt, 'Bruksareal'] = df.loc[~filt].apply(lambda x: x['Primærrom'] if 'm²' not in x['Bruksareal'] else x['Bruksareal'], axis = 1)

In [40]:
df['Bruksareal'] = df['Bruksareal'].apply(lambda x: x.replace(' (eiet)', ''))

#### Etasje

In [41]:
df['Etasje'].unique()

array(['1940', '1', '2016', '4', '5', nan, '1869', '3', '2', '10', '2002',
       '1898', '6', '8', '1952', '1988', '1899', '11', '1888', '1970',
       '1978', '1924', '1984', '1974', '1977', '1917', 1, '1919', '1956',
       '1953', '2020', '1929', '2014', '1960', '1991', '1990', '1986',
       '1963', '1983', '7', '-3', '12', '2013', '1890', '2021', '9',
       '1922', '-4', '1961', '1987', '1936', '1935', '1980', '1950',
       '1955', '1948', '1993', '-2', '1962', '1971', '1938', '1973',
       '1905', '1900', '1959', '1972', '2012', '1931', '1954', '1876',
       '1968', '1981', '2008', '1912', '1908', '1958', '2004', '1969',
       '1975', '1928', '1957', '1933', '1932', '1942', '1989', '1878',
       '2003', '1947', '1918', '2007', '1925', '1897', '1916', '1895'],
      dtype=object)

In [42]:
filt = ((df['Etasje'].astype(float) > 10) | (df['Etasje'].astype(float) < 0))
df.loc[filt]

,Address,Prisantydning,Fellesgjeld,Omkostninger,Totalpris,Felleskost,Boligtype,Eieform_bolig,Soverom,Primærrom,Bruksareal,Etasje,Byggeår,Energimerking,Beskrivelse,Eiendomsmegler
0,"St. Halvards gate 25E, 0192 Oslo",3850000.0,0.0,76000.0,3926000.0,3964.00,Leilighet,Selveier,1,48 m²,48 m²,1940,E - gul,NaN,Lys og lekker leilighet med solrik balkong. Go...,Aktiv Grønland/Bjørvika
2,"Gregers Grams vei 6C, 0382 Oslo",18250000.0,0.0,471920.0,18721920.0,2167.00,Enebolig,Selveier,4,208 m²,242 m²,2016,B - gul,NaN,Tiltalende og moderne kjedet enebolig m/ takte...,PrivatMegleren Dyve & Partnere
6,"Langes gate 7, 0165 Oslo",6500000.0,0.0,174070.0,6674070.0,2348.00,Leilighet,Selveier,2,82 m²,85 m²,1869,G - rød,NaN,"Klassisk, gjennomgående og sentrumsnær 3-roms ...",Privatmegleren Tjuvholmen
15,"Oskar Braatens gate 2, 0474 Oslo",4550000.0,424255.0,8010.0,4982265.0,4982265.00,Leilighet,Andel,2,69 m²,69 m²,2002,D - rød,NaN,Innbydende 3(4)-roms leilighet - *Garasjeleie ...,EIE eiendomsmegling Wekre og Partnere
23,"Rathkes gate 12C, 0558 Oslo",5990000.0,6945.0,165570.0,6162515.0,6162515.00,Tomannsbolig,Selveier,1,99 m²,99 m²,1898,C - rød,NaN,Stilig halvpart av opprinnelig stallbygg som e...,PrivatMegleren Première
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
900,"Eterveien 9, 0690 Oslo",8200000.0,72420.0,15510.0,8287930.0,8287930.00,Rekkehus,Andel,4,116 m²,127 m²,1970,E - gul,NaN,Familievennlig enderekkehus på Bøler i fantast...,PrivatMegleren Première
902,"Bergslia 10, 0854 Oslo",14900000.0,0.0,388170.0,15288170.0,1293.58,Enebolig,Selveier,3,138 m²,192 m²,1970,E - gul,NaN,BERG - Frittliggende enebolig med stor og frod...,PrivatMegleren Renommé
903,"Holmenkollveien 16 B, 0376 Oslo",19600000.0,0.0,506020.0,20106020.0,2850.33,Enebolig,Selveier,6,349 m²,353 m²,1895,G - gul,NaN,Stor og oppgradert enebolig - Pent opparbeidet...,"EIE eiendomsmegling Røa, Skøyen og Ullern"
908,"Monolitveien 14 C, 0369 Oslo",24700000.0,0.0,618878.0,25318878.0,3846.67,Enebolig,Selveier,4,315 m²,355 m²,1936,9,NaN,BORGEN: Flott enebolig o/3 etasjer + loft. Idy...,BOA Eiendomsmegling Oslo Vest AS


In [43]:
df.loc[filt, 'Energimerking'] = df.loc[filt].apply(lambda x: x['Byggeår'] if '-' in x['Byggeår'] else x['Energimerking'], axis = 1)

In [44]:
filt = (df['Byggeår'].str.contains('-'))
df.loc[filt]

,Address,Prisantydning,Fellesgjeld,Omkostninger,Totalpris,Felleskost,Boligtype,Eieform_bolig,Soverom,Primærrom,Bruksareal,Etasje,Byggeår,Energimerking,Beskrivelse,Eiendomsmegler
0,"St. Halvards gate 25E, 0192 Oslo",3850000.0,0.0,76000.0,3926000.0,3964.00,Leilighet,Selveier,1,48 m²,48 m²,1940,E - gul,E - gul,Lys og lekker leilighet med solrik balkong. Go...,Aktiv Grønland/Bjørvika
2,"Gregers Grams vei 6C, 0382 Oslo",18250000.0,0.0,471920.0,18721920.0,2167.00,Enebolig,Selveier,4,208 m²,242 m²,2016,B - gul,B - gul,Tiltalende og moderne kjedet enebolig m/ takte...,PrivatMegleren Dyve & Partnere
6,"Langes gate 7, 0165 Oslo",6500000.0,0.0,174070.0,6674070.0,2348.00,Leilighet,Selveier,2,82 m²,85 m²,1869,G - rød,G - rød,"Klassisk, gjennomgående og sentrumsnær 3-roms ...",Privatmegleren Tjuvholmen
15,"Oskar Braatens gate 2, 0474 Oslo",4550000.0,424255.0,8010.0,4982265.0,4982265.00,Leilighet,Andel,2,69 m²,69 m²,2002,D - rød,D - rød,Innbydende 3(4)-roms leilighet - *Garasjeleie ...,EIE eiendomsmegling Wekre og Partnere
23,"Rathkes gate 12C, 0558 Oslo",5990000.0,6945.0,165570.0,6162515.0,6162515.00,Tomannsbolig,Selveier,1,99 m²,99 m²,1898,C - rød,C - rød,Stilig halvpart av opprinnelig stallbygg som e...,PrivatMegleren Première
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
898,"Melkeveien 23A, 0779 Oslo",34800000.0,0.0,885670.0,35685670.0,2391.33,Enebolig,Selveier,6,460 m²,606 m²,1988,E - rød,E - rød,Praktfull villa med nydelig beliggenhet i Hemi...,PrivatMegleren Dyve & Partnere
900,"Eterveien 9, 0690 Oslo",8200000.0,72420.0,15510.0,8287930.0,8287930.00,Rekkehus,Andel,4,116 m²,127 m²,1970,E - gul,E - gul,Familievennlig enderekkehus på Bøler i fantast...,PrivatMegleren Première
902,"Bergslia 10, 0854 Oslo",14900000.0,0.0,388170.0,15288170.0,1293.58,Enebolig,Selveier,3,138 m²,192 m²,1970,E - gul,E - gul,BERG - Frittliggende enebolig med stor og frod...,PrivatMegleren Renommé
903,"Holmenkollveien 16 B, 0376 Oslo",19600000.0,0.0,506020.0,20106020.0,2850.33,Enebolig,Selveier,6,349 m²,353 m²,1895,G - gul,G - gul,Stor og oppgradert enebolig - Pent opparbeidet...,"EIE eiendomsmegling Røa, Skøyen og Ullern"


In [45]:
df.loc[filt, ['Energimerking', 'Byggeår']] = df.loc[filt, ['Byggeår', 'Etasje']].values

In [46]:
df.loc[filt,'Etasje'] =  df.loc[filt,'Etasje'].apply(lambda x: x if int(x) < 10 else np.NaN)

#### Byggeår

In [47]:
df['Byggeår'].unique()

array(['1940', '1953', '2016', '1958', '2005', '1990', '1869', '1991',
       '2018', '1946', '1892', '1880', '2019', '1959', '1957', '2002',
       '2008', '1952', '1960', '1961', '1893', '1903', '1898', '1894',
       '4', '1933', '1967', '1963', '2006', '1972', '3', '2007', '2012',
       '1', '1955', '1988', '1906', '1943', '1935', '1922', '1954',
       '1970', '1975', '1900', '2009', '1888', '1982', '1981', '1971',
       '1997', '1896', '1950', '1978', '1932', '1890', '1938', '1924',
       '1889', '1984', '1974', '2017', '1897', '1939', '1994', '1987',
       '1977', '1992', '9', '1956', '1973', '1918', '1919',
       '1522 m² (eiet)', '1926', '2020', '1929', '2021', '1996', '1989',
       '2014', '1979', '2015', '1881', '1930', '1965', '2013', '1969',
       '1986', '1928', '1931', '1936', '1899', '1983', '2000', '1966',
       '1951', '1980', '2011', '1901', '1937', '1934', '1907',
       '1054 m² (eiet)', '5', '2003', '1941', '1948', '1895', '1874',
       '1968', '1875', '1

In [48]:
filt = (df['Byggeår'].str.contains('m²'))
df.loc[filt]

,Address,Prisantydning,Fellesgjeld,Omkostninger,Totalpris,Felleskost,Boligtype,Eieform_bolig,Soverom,Primærrom,Bruksareal,Etasje,Byggeår,Energimerking,Beskrivelse,Eiendomsmegler
114,"Vøyensvingen 16D, 0458 Oslo",3400000.0,288553.0,8032.0,3696585.0,3696585.00,Leilighet,Andel,1,29 m²,29 m²,1953,1522 m² (eiet),NaN,Pen 2-roms andel med balkong i frodig område -...,PrivatMegleren Park
202,"Lambrechts gate 8, 0262 Oslo",19900000.0,333517.0,517362.0,20750879.0,20750879.00,Leilighet,Selveier,3,208 m²,208 m²,1890,1054 m² (eiet),NaN,"Nyoppusset i 21! Eksklusiv leilighet i flott, ...",Aktiv Grünerløkka
426,"Guldbergs vei 38, 0375 Oslo",14950000.0,0.0,389420.0,15339420.0,2000.00,Tomannsbolig,Selveier,4,130 m²,132 m²,2014,1312 m² (eiet),NaN,Nyere (2014) familiebolig med solrik takterras...,PrivatMegleren Allé
641,"Fiskekroken 12C, 0139 Oslo",21000000.0,0.0,468520.0,21468520.0,1916.67,Enebolig,Selveier,4,310 m²,326 m²,1973,1204 m² (eiet),NaN,SJELDEN MULIGHET: Attraktiv eiendom tett på Os...,EIE eiendomsmegling Bekkelaget & Nordstrand
667,"Holmenkollveien 33B, 0376 Oslo",17900000.0,0.0,463170.0,18363170.0,0.00,Enebolig,Selveier,4,218 m²,260 m²,1942,1409 m² (eiet),NaN,Enebolig på stor solrik tomt i et veletablert ...,PrivatMegleren Dyve & Partnere


In [49]:
df.loc[filt, 'Byggeår'] = df.loc[filt, 'Etasje'].values

In [50]:
df.loc[filt, 'Etasje'] = np.NaN

In [51]:
filt = (df['Byggeår'].astype(int) < 10)
df.loc[filt]

,Address,Prisantydning,Fellesgjeld,Omkostninger,Totalpris,Felleskost,Boligtype,Eieform_bolig,Soverom,Primærrom,Bruksareal,Etasje,Byggeår,Energimerking,Beskrivelse,Eiendomsmegler
27,"Ensjøveien 4, 0655 Oslo",3190000.0,5740000.0,81159.0,9011159.0,9011159.00,Leilighet,Andel,3,87 m²,87 m²,4,4,NaN,Ny 4-roms toppleilighet med 2 balkonger | 2 ba...,Schala & Partners Grünerløkka
36,"Bentsebrugata 13B, 0476 Oslo",4700000.0,0.0,118874.0,4818874.0,2905.00,Leilighet,Selveier,2,56 m²,59 m²,1952,3,NaN,Pen 3-roms leilighet over 2 plan med sentral b...,EiendomsMegler 1 Torshov
39,"Hesselbergs gate 8 A, 0555 Oslo",3100000.0,182761.0,1167.0,3283928.0,3283928.00,Leilighet,Andel,1,30 m²,30 m²,1898,1,NaN,GRÜNERLØKKA: Lys og luftig med praktisk alkove...,Boa Eiendomsmegling Oslo Øst
52,"Sofies gate 88A, 0454 Oslo",7790000.0,255762.0,202462.0,8248224.0,8248224.00,Leilighet,Selveier,2,115 m²,115 m²,1899,3,NaN,Klassisk og gjennomgående 4-roms hjørneleiligh...,Nordvik Majorstuen
97,"Drammensveien 103 D og E, 0273 Oslo",45500000.0,0.0,1138878.0,46638878.0,1893.33,Enebolig,Selveier,6,380 m²,677 m²,1917,9,NaN,FROGNER/SKARPSNO: Herskapelig villaeiendom med...,BOA Eiendomsmegling Oslo Vest AS
144,"Ullevålsveien 81, 0454 Oslo",6950000.0,148000.0,19248.0,7117248.0,7117248.00,Leilighet,Aksje,3,95 m²,95 m²,1,4,NaN,Adamstuen: Stor og gjennomgående 4-roms leilig...,Aktiv Torshov
165,"Rødtvetveien 16, 0955 Oslo",2750000.0,42442.0,8060.0,2800502.0,2800502.00,Leilighet,Andel,5,45 m²,35 m²,1963,1,NaN,Arealeffektiv 1-roms med alkove - Innglasset b...,NaN
203,"Th. Kittelsens vei 26, 0379 Oslo",13250000.0,0.0,93342.0,13343342.0,0.00,Tomannsbolig,Selveier,4,143 m²,180 m²,2021,5,NaN,Abbedikollen - Attraktiv halvpart tomannsbolig...,Nordvik Bygdøy Allé
227,"Skogfaret 23, 0382 Oslo",28000000.0,0.0,701342.0,28701342.0,1433.58,Enebolig,Selveier,7,190 m²,220 m²,1956,8,NaN,Innholdsrik enebolig over to plan. Solrik tomt...,NaN
303,"Langviksveien 19C, 0286 Oslo",13900000.0,0.0,348874.0,14248874.0,922.50,Rekkehus,Selveier,3,123 m²,125 m²,1974,6,NaN,Nydelig og sjeldent rekkehus med 3(4) soverom ...,EiendomsMegler 1 Torshov


In [52]:
# the 'Beskrivelse' column is supporting that, these values belongs to the 'Soverom' column. 

df.loc[filt, 'Soverom'] = df.loc[filt, 'Byggeår'].values 

In [53]:
df.loc[filt, 'Byggeår'] = df.loc[filt, 'Etasje'].apply(lambda x: x if int(x) > 10 else np.NaN)

In [54]:
df.loc[filt]

,Address,Prisantydning,Fellesgjeld,Omkostninger,Totalpris,Felleskost,Boligtype,Eieform_bolig,Soverom,Primærrom,Bruksareal,Etasje,Byggeår,Energimerking,Beskrivelse,Eiendomsmegler
27,"Ensjøveien 4, 0655 Oslo",3190000.0,5740000.0,81159.0,9011159.0,9011159.00,Leilighet,Andel,4,87 m²,87 m²,4,NaN,NaN,Ny 4-roms toppleilighet med 2 balkonger | 2 ba...,Schala & Partners Grünerløkka
36,"Bentsebrugata 13B, 0476 Oslo",4700000.0,0.0,118874.0,4818874.0,2905.00,Leilighet,Selveier,3,56 m²,59 m²,1952,1952,NaN,Pen 3-roms leilighet over 2 plan med sentral b...,EiendomsMegler 1 Torshov
39,"Hesselbergs gate 8 A, 0555 Oslo",3100000.0,182761.0,1167.0,3283928.0,3283928.00,Leilighet,Andel,1,30 m²,30 m²,1898,1898,NaN,GRÜNERLØKKA: Lys og luftig med praktisk alkove...,Boa Eiendomsmegling Oslo Øst
52,"Sofies gate 88A, 0454 Oslo",7790000.0,255762.0,202462.0,8248224.0,8248224.00,Leilighet,Selveier,3,115 m²,115 m²,1899,1899,NaN,Klassisk og gjennomgående 4-roms hjørneleiligh...,Nordvik Majorstuen
97,"Drammensveien 103 D og E, 0273 Oslo",45500000.0,0.0,1138878.0,46638878.0,1893.33,Enebolig,Selveier,9,380 m²,677 m²,1917,1917,NaN,FROGNER/SKARPSNO: Herskapelig villaeiendom med...,BOA Eiendomsmegling Oslo Vest AS
144,"Ullevålsveien 81, 0454 Oslo",6950000.0,148000.0,19248.0,7117248.0,7117248.00,Leilighet,Aksje,4,95 m²,95 m²,1,NaN,NaN,Adamstuen: Stor og gjennomgående 4-roms leilig...,Aktiv Torshov
165,"Rødtvetveien 16, 0955 Oslo",2750000.0,42442.0,8060.0,2800502.0,2800502.00,Leilighet,Andel,1,45 m²,35 m²,1963,1963,NaN,Arealeffektiv 1-roms med alkove - Innglasset b...,NaN
203,"Th. Kittelsens vei 26, 0379 Oslo",13250000.0,0.0,93342.0,13343342.0,0.00,Tomannsbolig,Selveier,5,143 m²,180 m²,2021,2021,NaN,Abbedikollen - Attraktiv halvpart tomannsbolig...,Nordvik Bygdøy Allé
227,"Skogfaret 23, 0382 Oslo",28000000.0,0.0,701342.0,28701342.0,1433.58,Enebolig,Selveier,8,190 m²,220 m²,1956,1956,NaN,Innholdsrik enebolig over to plan. Solrik tomt...,NaN
303,"Langviksveien 19C, 0286 Oslo",13900000.0,0.0,348874.0,14248874.0,922.50,Rekkehus,Selveier,6,123 m²,125 m²,1974,1974,NaN,Nydelig og sjeldent rekkehus med 3(4) soverom ...,EiendomsMegler 1 Torshov


In [55]:
df.loc[filt, 'Etasje'] = df.loc[filt, 'Etasje'].apply(lambda x: x if int(x) <= 10 else np.NaN )

In [56]:
df.loc[filt]

,Address,Prisantydning,Fellesgjeld,Omkostninger,Totalpris,Felleskost,Boligtype,Eieform_bolig,Soverom,Primærrom,Bruksareal,Etasje,Byggeår,Energimerking,Beskrivelse,Eiendomsmegler
27,"Ensjøveien 4, 0655 Oslo",3190000.0,5740000.0,81159.0,9011159.0,9011159.00,Leilighet,Andel,4,87 m²,87 m²,4,NaN,NaN,Ny 4-roms toppleilighet med 2 balkonger | 2 ba...,Schala & Partners Grünerløkka
36,"Bentsebrugata 13B, 0476 Oslo",4700000.0,0.0,118874.0,4818874.0,2905.00,Leilighet,Selveier,3,56 m²,59 m²,NaN,1952,NaN,Pen 3-roms leilighet over 2 plan med sentral b...,EiendomsMegler 1 Torshov
39,"Hesselbergs gate 8 A, 0555 Oslo",3100000.0,182761.0,1167.0,3283928.0,3283928.00,Leilighet,Andel,1,30 m²,30 m²,NaN,1898,NaN,GRÜNERLØKKA: Lys og luftig med praktisk alkove...,Boa Eiendomsmegling Oslo Øst
52,"Sofies gate 88A, 0454 Oslo",7790000.0,255762.0,202462.0,8248224.0,8248224.00,Leilighet,Selveier,3,115 m²,115 m²,NaN,1899,NaN,Klassisk og gjennomgående 4-roms hjørneleiligh...,Nordvik Majorstuen
97,"Drammensveien 103 D og E, 0273 Oslo",45500000.0,0.0,1138878.0,46638878.0,1893.33,Enebolig,Selveier,9,380 m²,677 m²,NaN,1917,NaN,FROGNER/SKARPSNO: Herskapelig villaeiendom med...,BOA Eiendomsmegling Oslo Vest AS
144,"Ullevålsveien 81, 0454 Oslo",6950000.0,148000.0,19248.0,7117248.0,7117248.00,Leilighet,Aksje,4,95 m²,95 m²,1,NaN,NaN,Adamstuen: Stor og gjennomgående 4-roms leilig...,Aktiv Torshov
165,"Rødtvetveien 16, 0955 Oslo",2750000.0,42442.0,8060.0,2800502.0,2800502.00,Leilighet,Andel,1,45 m²,35 m²,NaN,1963,NaN,Arealeffektiv 1-roms med alkove - Innglasset b...,NaN
203,"Th. Kittelsens vei 26, 0379 Oslo",13250000.0,0.0,93342.0,13343342.0,0.00,Tomannsbolig,Selveier,5,143 m²,180 m²,NaN,2021,NaN,Abbedikollen - Attraktiv halvpart tomannsbolig...,Nordvik Bygdøy Allé
227,"Skogfaret 23, 0382 Oslo",28000000.0,0.0,701342.0,28701342.0,1433.58,Enebolig,Selveier,8,190 m²,220 m²,NaN,1956,NaN,Innholdsrik enebolig over to plan. Solrik tomt...,NaN
303,"Langviksveien 19C, 0286 Oslo",13900000.0,0.0,348874.0,14248874.0,922.50,Rekkehus,Selveier,6,123 m²,125 m²,NaN,1974,NaN,Nydelig og sjeldent rekkehus med 3(4) soverom ...,EiendomsMegler 1 Torshov


#### Energimerking

In [57]:
# not to much to do with 'Energimerking' for now, except for filling the missing values.

df['Energimerking'].unique()

array(['E - gul', 'G - rød', 'B - gul', nan, 'C - mørkegrønn',
       'F - mørkegrønn', 'E - mørkegrønn', 'G - oransje', 'D - lysegrønn',
       'D - rød', 'D - mørkegrønn', 'G - mørkegrønn', 'C - gul',
       'C - rød', 'D - gul', 'F - rød', 'G - lysegrønn', 'F - oransje',
       'B - mørkegrønn', 'E - rød', 'F - gul', 'E - oransje',
       'E - lysegrønn', 'C - oransje', 'D - oransje', 'B - oransje',
       'G - gul', 'C - lysegrønn', 'A - gul', 'A - lysegrønn',
       'B - lysegrønn', 'F - lysegrønn', 'B - rød'], dtype=object)

In [58]:
df['Boligtype'].value_counts()

Leilighet       802
Enebolig         66
Tomannsbolig     33
Rekkehus         23
Name: Boligtype, dtype: int64

#### Etasje again

In [59]:
# get all rows where 'Etasje' is missing. 

filt = (df['Etasje'].isna())
df.loc[filt]

,Address,Prisantydning,Fellesgjeld,Omkostninger,Totalpris,Felleskost,Boligtype,Eieform_bolig,Soverom,Primærrom,Bruksareal,Etasje,Byggeår,Energimerking,Beskrivelse,Eiendomsmegler
0,"St. Halvards gate 25E, 0192 Oslo",3850000.0,0.0,76000.0,3926000.0,3964.00,Leilighet,Selveier,1,48 m²,48 m²,NaN,1940,E - gul,Lys og lekker leilighet med solrik balkong. Go...,Aktiv Grønland/Bjørvika
2,"Gregers Grams vei 6C, 0382 Oslo",18250000.0,0.0,471920.0,18721920.0,2167.00,Enebolig,Selveier,4,208 m²,242 m²,NaN,2016,B - gul,Tiltalende og moderne kjedet enebolig m/ takte...,PrivatMegleren Dyve & Partnere
5,"Kantarellen terrasse 15, 1286 Oslo",2590000.0,216310.0,8658.0,2814968.0,2814968.00,Leilighet,Andel,1,59 m²,59 m²,NaN,1990,F - mørkegrønn,Stilig 2-roms med bad fra 2019 og garasjeplass...,Eiendomsmegler 1 Oslo Syd
6,"Langes gate 7, 0165 Oslo",6500000.0,0.0,174070.0,6674070.0,2348.00,Leilighet,Selveier,2,82 m²,85 m²,NaN,1869,G - rød,"Klassisk, gjennomgående og sentrumsnær 3-roms ...",Privatmegleren Tjuvholmen
15,"Oskar Braatens gate 2, 0474 Oslo",4550000.0,424255.0,8010.0,4982265.0,4982265.00,Leilighet,Andel,2,69 m²,69 m²,NaN,2002,D - rød,Innbydende 3(4)-roms leilighet - *Garasjeleie ...,EIE eiendomsmegling Wekre og Partnere
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
900,"Eterveien 9, 0690 Oslo",8200000.0,72420.0,15510.0,8287930.0,8287930.00,Rekkehus,Andel,4,116 m²,127 m²,NaN,1970,E - gul,Familievennlig enderekkehus på Bøler i fantast...,PrivatMegleren Première
902,"Bergslia 10, 0854 Oslo",14900000.0,0.0,388170.0,15288170.0,1293.58,Enebolig,Selveier,3,138 m²,192 m²,NaN,1970,E - gul,BERG - Frittliggende enebolig med stor og frod...,PrivatMegleren Renommé
903,"Holmenkollveien 16 B, 0376 Oslo",19600000.0,0.0,506020.0,20106020.0,2850.33,Enebolig,Selveier,6,349 m²,353 m²,NaN,1895,G - gul,Stor og oppgradert enebolig - Pent opparbeidet...,"EIE eiendomsmegling Røa, Skøyen og Ullern"
908,"Monolitveien 14 C, 0369 Oslo",24700000.0,0.0,618878.0,25318878.0,3846.67,Enebolig,Selveier,9,315 m²,355 m²,NaN,1936,NaN,BORGEN: Flott enebolig o/3 etasjer + loft. Idy...,BOA Eiendomsmegling Oslo Vest AS


In [60]:
# If the 'Boligtype' for these is one of these ['Enebolig', 'Rekkehus' or 'Tomannsbolig'], then
# I will fill the missing value with 1. 
# If the 'Boligtype' is 'Leilighet' I will then fill the missing value with the most frequent value, which is the second floor '2'

l = ['Enebolig', 'Rekkehus', 'Tomannsbolig']
df.loc[filt, 'Etasje'] = df.loc[filt].apply(lambda x: '1' if x['Boligtype'] in l else '2', axis = 1)

In [61]:
df.loc[filt]

,Address,Prisantydning,Fellesgjeld,Omkostninger,Totalpris,Felleskost,Boligtype,Eieform_bolig,Soverom,Primærrom,Bruksareal,Etasje,Byggeår,Energimerking,Beskrivelse,Eiendomsmegler
0,"St. Halvards gate 25E, 0192 Oslo",3850000.0,0.0,76000.0,3926000.0,3964.00,Leilighet,Selveier,1,48 m²,48 m²,2,1940,E - gul,Lys og lekker leilighet med solrik balkong. Go...,Aktiv Grønland/Bjørvika
2,"Gregers Grams vei 6C, 0382 Oslo",18250000.0,0.0,471920.0,18721920.0,2167.00,Enebolig,Selveier,4,208 m²,242 m²,1,2016,B - gul,Tiltalende og moderne kjedet enebolig m/ takte...,PrivatMegleren Dyve & Partnere
5,"Kantarellen terrasse 15, 1286 Oslo",2590000.0,216310.0,8658.0,2814968.0,2814968.00,Leilighet,Andel,1,59 m²,59 m²,2,1990,F - mørkegrønn,Stilig 2-roms med bad fra 2019 og garasjeplass...,Eiendomsmegler 1 Oslo Syd
6,"Langes gate 7, 0165 Oslo",6500000.0,0.0,174070.0,6674070.0,2348.00,Leilighet,Selveier,2,82 m²,85 m²,2,1869,G - rød,"Klassisk, gjennomgående og sentrumsnær 3-roms ...",Privatmegleren Tjuvholmen
15,"Oskar Braatens gate 2, 0474 Oslo",4550000.0,424255.0,8010.0,4982265.0,4982265.00,Leilighet,Andel,2,69 m²,69 m²,2,2002,D - rød,Innbydende 3(4)-roms leilighet - *Garasjeleie ...,EIE eiendomsmegling Wekre og Partnere
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
900,"Eterveien 9, 0690 Oslo",8200000.0,72420.0,15510.0,8287930.0,8287930.00,Rekkehus,Andel,4,116 m²,127 m²,1,1970,E - gul,Familievennlig enderekkehus på Bøler i fantast...,PrivatMegleren Première
902,"Bergslia 10, 0854 Oslo",14900000.0,0.0,388170.0,15288170.0,1293.58,Enebolig,Selveier,3,138 m²,192 m²,1,1970,E - gul,BERG - Frittliggende enebolig med stor og frod...,PrivatMegleren Renommé
903,"Holmenkollveien 16 B, 0376 Oslo",19600000.0,0.0,506020.0,20106020.0,2850.33,Enebolig,Selveier,6,349 m²,353 m²,1,1895,G - gul,Stor og oppgradert enebolig - Pent opparbeidet...,"EIE eiendomsmegling Røa, Skøyen og Ullern"
908,"Monolitveien 14 C, 0369 Oslo",24700000.0,0.0,618878.0,25318878.0,3846.67,Enebolig,Selveier,9,315 m²,355 m²,1,1936,NaN,BORGEN: Flott enebolig o/3 etasjer + loft. Idy...,BOA Eiendomsmegling Oslo Vest AS


In [62]:
df.isna().sum()

Address             0
Prisantydning       0
Fellesgjeld         0
Omkostninger        0
Totalpris           0
Felleskost          0
Boligtype           0
Eieform_bolig       0
Soverom             0
Primærrom           0
Bruksareal          0
Etasje              0
Byggeår             5
Energimerking     109
Beskrivelse         0
Eiendomsmegler    198
dtype: int64

#### Byggeår igjen

In [63]:
filt = (df['Byggeår'].isna())
df.loc[filt]

,Address,Prisantydning,Fellesgjeld,Omkostninger,Totalpris,Felleskost,Boligtype,Eieform_bolig,Soverom,Primærrom,Bruksareal,Etasje,Byggeår,Energimerking,Beskrivelse,Eiendomsmegler
27,"Ensjøveien 4, 0655 Oslo",3190000.0,5740000.0,81159.0,9011159.0,9011159.0,Leilighet,Andel,4,87 m²,87 m²,4,NaN,NaN,Ny 4-roms toppleilighet med 2 balkonger | 2 ba...,Schala & Partners Grünerløkka
144,"Ullevålsveien 81, 0454 Oslo",6950000.0,148000.0,19248.0,7117248.0,7117248.0,Leilighet,Aksje,4,95 m²,95 m²,1,NaN,NaN,Adamstuen: Stor og gjennomgående 4-roms leilig...,Aktiv Torshov
462,"Ammerudveien 68, 0958 Oslo",3390000.0,409000.0,15504.0,3814504.0,3814504.0,Leilighet,Andel,3,83 m²,93 m²,3,NaN,E - mørkegrønn,Attraktiv 4-roms med vestvendt balkong og mark...,EIE eiendomsmegling Stovner
685,"Aslakveien 17C, 0753 Oslo",4700000.0,183303.0,131090.0,5014393.0,5014393.0,Leilighet,Selveier,3,66 m²,66 m²,3,NaN,G - mørkegrønn,Attraktiv 3(4)-roms leilighet m/balkong. Fanta...,EIE eiendomsmegling Lillestrøm
802,"Locus ensjø Leil. D2-305, 0661 Oslo",4950000.0,0.0,70834.0,5020834.0,2750.0,Leilighet,Selveier,2,46 m²,50 m²,3,NaN,NaN,LOCUS ENSJØ - Stor og lys 2-roms med vestvendt...,Aktiv Carl Berner


In [64]:
for idx in df.loc[filt].index:
     print(idx,':', df.loc[idx, 'Beskrivelse'])

27 : Ny 4-roms toppleilighet med 2 balkonger | 2 bad | Garasjeplass | Ferdigstilles Q3/Q4 2023 | Salg av kontraktsposisjon
144 : Adamstuen: Stor og gjennomgående 4-roms leilighet med 3 soverom og god planløsning. Ikke dok. avgift eller forkjøpsrett.
462 : Attraktiv 4-roms med vestvendt balkong og marka som nærmeste nabo- Varmtvann og fyring inkludert - Heis - Må sees!
685 : Attraktiv 3(4)-roms leilighet m/balkong. Fantastisk utsikt. Lys og luftig. Fyring og vv. inkl.
802 : LOCUS ENSJØ - Stor og lys 2-roms med vestvendt balkong på 9m². Garasjeplass inkl. Innflytting Q1 2023. Salg av kontrakt.


In [65]:
# because these 3 Boliger are new, so i will set 'Byggeår' to 2021
df.loc[27, 'Byggeår'] = '2021'
df.loc[802, 'Byggeår'] = '2021'

In [66]:
# I checked these appartments on finn.no and the 'Byggeår' wasn't mentioned.
# I will fill the other 3 Boliger with '1950' (the best what I came with).

df['Byggeår'].fillna('1950', inplace = True)

#### Eiendomsmegler

In [67]:
df['Eiendomsmegler'].value_counts()
temp_df = pd.DataFrame(df['Eiendomsmegler'].value_counts()) 
temp_df.head()

,Eiendomsmegler
PrivatMegleren Panorama,30
Sem & Johnsen,28
Nordvik Frogner,24
PrivatMegleren Dyve & Partnere,23
PrivatMegleren Park,21


In [68]:
temp_df.head(15)

,Eiendomsmegler
PrivatMegleren Panorama,30
Sem & Johnsen,28
Nordvik Frogner,24
PrivatMegleren Dyve & Partnere,23
PrivatMegleren Park,21
Nordvik Bygdøy Allé,20
Nordvik Majorstuen,18
Aktiv Carl Berner,18
"EIE eiendomsmegling Røa, Skøyen og Ullern",16
Notar Oslo,15


In [69]:
def generalize_eiendomsmegler(x):
    if ('PrivatMegleren' in x) or ('Privatmegleren' in x):
        return 'PrivatMegleren'
    elif ('Eiendomsmegler 1' in x) or ('EiendomsMegler 1' in x):
        return 'Eiendomsmegler 1'
    elif ('BOA' in x) or ('Boa' in x) :
        return 'BOA Eiendomsmegling'
    elif 'Nordvik' in x:
        return 'Nordvik'
    elif 'Aktiv' in x:
        return 'Aktiv'
    elif 'EIE eiendomsmegling' in x:
        return 'EIE eiendomsmegling'
    elif 'Notar' in x:
        return 'Notar'
    elif 'Krogsveen' in x:
        return 'Krogsveen'
    elif 'Schala & Partners' in x:
        return 'Schala & Partners'
    elif 'DNB' in x:
        return 'DNB Eiendom'
    elif 'Privatmegleren' in x:
        return 'Privatmegleren'
    else:
        return x

In [70]:
filt = (df['Eiendomsmegler'].isna())
df.loc[~filt]

,Address,Prisantydning,Fellesgjeld,Omkostninger,Totalpris,Felleskost,Boligtype,Eieform_bolig,Soverom,Primærrom,Bruksareal,Etasje,Byggeår,Energimerking,Beskrivelse,Eiendomsmegler
0,"St. Halvards gate 25E, 0192 Oslo",3850000.0,0.0,76000.0,3926000.0,3964.0,Leilighet,Selveier,1,48 m²,48 m²,2,1940,E - gul,Lys og lekker leilighet med solrik balkong. Go...,Aktiv Grønland/Bjørvika
1,"Selvbyggerveien 143, 0591 Oslo",3200000.0,73058.0,8009.0,3281067.0,3281067.0,Leilighet,Andel,1,35 m²,37 m²,1,1953,G - rød,Romslig og attraktiv 2-roms | Hyggelig utsikt ...,Schala & Partners Årvoll
2,"Gregers Grams vei 6C, 0382 Oslo",18250000.0,0.0,471920.0,18721920.0,2167.0,Enebolig,Selveier,4,208 m²,242 m²,1,2016,B - gul,Tiltalende og moderne kjedet enebolig m/ takte...,PrivatMegleren Dyve & Partnere
3,"Brochmanns gate 7, 0470 Oslo",4600000.0,101699.0,127039.0,4828738.0,4828738.0,Leilighet,Selveier,2,54 m²,54 m²,4,1958,NaN,Lys og pen 3-R selveier med nydelig balkong I ...,Schala & Partners Sagene
5,"Kantarellen terrasse 15, 1286 Oslo",2590000.0,216310.0,8658.0,2814968.0,2814968.0,Leilighet,Andel,1,59 m²,59 m²,2,1990,F - mørkegrønn,Stilig 2-roms med bad fra 2019 og garasjeplass...,Eiendomsmegler 1 Oslo Syd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
916,"Stensgata 27, 0358 Oslo",8500000.0,0.0,19150.0,8519150.0,3255.0,Leilighet,Aksje,2,91 m²,91 m²,2,1923,E - oransje,Lekker 3-roms (91 kvm) med balkong - HTH kjøkk...,PrivatMegleren Scenario
919,"Bygdøy allé 70, 0268 Oslo",4500000.0,2212.0,122192.0,4624404.0,4624404.0,Leilighet,Selveier,1,52 m²,52 m²,3,1931,G - rød,Attraktiv og romslig 2-roms i 3. etasje med so...,Nordvik Bygdøy Allé
921,"Minister Ditleffs vei 5B, 0862 Oslo",2600000.0,57000.0,8658.0,2665658.0,2665658.0,Leilighet,Andel,0,33 m²,33 m²,1,1963,F - mørkegrønn,Arealeffektiv 1-roms oppussingsobjekt | Vestve...,EiendomsMegler 1 Torshov
922,"Mandalls gate 5B, 0190 Oslo",3000000.0,1011249.0,8032.0,4019281.0,4019281.0,Leilighet,Andel,2,50 m²,50 m²,3,1888,F - rød,Attraktiv 3-roms leilighet i 3 etg | Svært sen...,Nordvik Lillestrøm


In [71]:
df.loc[~filt, 'Eiendomsmegler'] = df.loc[~filt, 'Eiendomsmegler'].apply(generalize_eiendomsmegler)

In [72]:
df['Eiendomsmegler'].value_counts()

PrivatMegleren                          167
Nordvik                                 140
EIE eiendomsmegling                      79
Krogsveen                                79
Eiendomsmegler 1                         66
Aktiv                                    63
Schala & Partners                        46
Sem & Johnsen                            28
BOA Eiendomsmegling                      16
Notar                                    15
DNB Eiendom                               7
Propr.no                                  6
Røisland & Co Eiendomsmegling AS          5
Proaktiv Eiendomsmegling                  4
RE/MAX Zaar & Partnere                    2
Advokat Ek                                1
Estate Meglerne - Din Boligmegler AS      1
Advokatfirmaet Storløkken AS              1
Name: Eiendomsmegler, dtype: int64

#### Felleskost

In [74]:
filt = (df['Felleskost'] == df['Totalpris'])
df.loc[filt].shape[0]

628

In [75]:
# replace these values with the average (mean) of Felleskost. but first replace these values with NaN because if i don't the 
# high values of Felleskost column which actually belongs to Totalpris will have a big effect on the mean value of the column
df.loc[filt, 'Felleskost'] = np.NaN

In [ ]:
df['Felleskost'].fillna(np.round(df['Felleskost'].mean(), 2), inplace=True)

In [ ]:
# get all rows where Etasje contians any of these values 18,19,20
#century_lst = ['18','19','20']

#filt = (df['Etasje'].str.contains('|'.join(century_lst)))
#df.loc[filt]

### Feature engineering

#### Distance from the city center is very important when it comes to determining a house price. The distance from the city center is not methioned in the dataset, but I have the zipcode for each house in the dataset (from the address). So I will use google 'Distance Matrix API' to calculate the distance from 'Oslo train station' (my center point) to the house. I am using my own 'API Key' to run/reuse this code use your own.


In [ ]:
key = use_your_API_key_here
city_center = '0154 OSLO' 

def find_distance_from_center(x):
    destination = x.split(',')[1].strip()
    url = 'https://maps.googleapis.com/maps/api/distancematrix/json?origins={}&destinations={}&key={}'.format(city_center, destination, key)
    payload={}
    headers = {}
    response = requests.request("GET", url, headers=headers, data=payload).json()
    
    for obj in response['rows']:
    #print(obj)
        for data in obj['elements']:
        #print(data)
            return data['distance']['text']

In [ ]:
df['avstand'] = df['Address'].apply(find_distance_from_center)

In [ ]:
df['avstand']

In [ ]:
df['avstand'] = df['avstand'].str.replace(' km', '')

In [ ]:
# extract usuful features from Beskrivelse

from wordcloud import WordCloud, ImageColorGenerator, STOPWORDS
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


words = " ".join(df['Beskrivelse'])

def punctuation_stop(text):
    """remove punctuation and stop words"""
    filtered = []
    stop_words = set(stopwords.words('norwegian'))
    word_tokens = word_tokenize(text)
    for w in word_tokens:
        if w not in stop_words and w.isalpha():
            filtered.append(w.lower())
    return filtered


words_filtered = punctuation_stop(words)

text = " ".join([ele for ele in words_filtered])

wc= WordCloud(background_color="white", random_state=1,stopwords=STOPWORDS, max_words = 2000, width =800, height = 1500)
wc.generate(text)

plt.figure(figsize=[10,14])
plt.imshow(wc,interpolation="bilinear")
plt.axis('off')
plt.show()

In [ ]:
# Based on the above picture I choose to create features for 'balkong', 'utsikt', 'garasje', 
# 'heis','parkering','hage','toppleilighet'

In [ ]:
df['balkong'] = df['Beskrivelse'].apply(lambda x: 1 if 'balkong' in x.lower() else 0)

In [ ]:
df['balkong'].value_counts()

In [ ]:
df['utsikt'] = df['Beskrivelse'].apply(lambda x: 1 if 'utsikt' in x.lower() else 0)

In [ ]:
# Both rows that contains 'garasje' or 'garasjeplass' will be included here

df['garasje'] = df['Beskrivelse'].apply(lambda x: 1 if 'garasje' in x.lower() else 0)

In [ ]:
df['heis'] = df['Beskrivelse'].apply(lambda x: 1 if 'heis' in x.lower() else 0)

In [ ]:
df['parkering'] = df['Beskrivelse'].apply(lambda x: 1 if 'parkering' in x.lower() else 0)

In [ ]:
df['hage'] = df['Beskrivelse'].apply(lambda x: 1 if 'hage' in x.lower() else 0)

In [ ]:
# Both rows that contains 'sentrum' or 'sentralt' will be included here

df['sentrum'] = df['Beskrivelse'].apply(lambda x: 1 if any(s in x.lower() for s in ('sentrum', 'sentralt')) else 0)

In [ ]:
df['toppleilighet'] = df['Beskrivelse'].apply(lambda x: 1 if 'toppleilighet' in x.lower() else 0)

In [ ]:
df.head()

In [ ]:
# calculating the age of the Bolig

df['Alder'] = df['Byggeår'].apply(lambda x: 2021 - int(x))

In [ ]:
df['Primærrom'] = df['Primærrom'].apply(lambda x: x.replace(' m²',''))
df['Bruksareal'] = df['Bruksareal'].apply(lambda x: x.replace(' m²',''))

In [ ]:
df.head()

In [ ]:
# 'Prisantydning', 'Fellesgjeld', 'Omkostninger', 'Totalpris', 'Felleskost' form Categorical to Numrical

df[['Prisantydning', 'Fellesgjeld', 'Omkostninger', 'Totalpris', 'Felleskost','Primærrom','Bruksareal','Etasje','Soverom']] = df[['Prisantydning', 'Fellesgjeld', 'Omkostninger', 'Totalpris', 'Felleskost','Primærrom','Bruksareal','Etasje','Soverom']].apply(pd.to_numeric)

In [ ]:
df.info()

In [ ]:
df.columns

### dealing with missing Eiendomsmegler

In [ ]:
# I can't drop the rows with missing Eiendomsmegler, because this will lead to losing a big portion of the dataset
# I can't either fill it with whatever value
# I choose to treat it as if it was a real value

In [ ]:
#from sklearn.impute import SimpleImputer

In [ ]:
#new_cat = SimpleImputer(strategy = 'constant')

In [ ]:
#df['Eiendomsmegler'] = new_cat.fit_transform(df[['Eiendomsmegler']])

In [ ]:
#df['Eiendomsmegler'].isna().sum()

In [ ]:
df['Eiendomsmegler'].fillna('Ukjent', inplace = True)

In [ ]:
df['Eiendomsmegler'].value_counts()

In [ ]:
df['Eiendomsmegler'].unique()

In [ ]:
df.isna().sum().plot(kind = 'bar', figsize = (15,8))

In [ ]:
#df['Energimerking'].mode()

### dealing with missing Energimerking

In [ ]:
# I can drop the rows with missing Energimerking, but i don't have a large dataset so this is not a good practice
# I could fill the missing values with the "mode" which is the most frequently occurring value. 
# --> in Energimerking the mode is 'G - rød', but this can lead to unbalanced data 
# build a model to predict the missing Energimerking

In [ ]:
# the order here matters, means that 'A' is the best Energimerking and 'G - rød' is the worst
df['Energimerking'].value_counts().index.sort_values( ascending = False)

In [ ]:
# since the order matters I will use Ordinal Encoding for converting these values into numerical values
Energimerking_enco = { 'G - rød':1, 'G - oransje':2, 'G - mørkegrønn':3, 'G - lysegrønn':4, 'G - gul':5,
       'F - rød':6, 'F - oransje':7, 'F - mørkegrønn':8, 'F - lysegrønn':9, 'F - gul':10,
       'E - rød':10, 'E - oransje':11, 'E - mørkegrønn':12, 'E - lysegrønn':13, 'E - gul':14,
       'D - rød':15, 'D - oransje':16, 'D - mørkegrønn':17, 'D - lysegrønn':18, 'D - gul':19,
       'C - rød':20, 'C - oransje':21, 'C - mørkegrønn':22, 'C - lysegrønn':23, 'C - gul':24,
       'B - rød':25, 'B - oransje':26, 'B - mørkegrønn':27, 'B - lysegrønn':28, 'B - gul':29,
       'A - lysegrønn':30, 'A - gul':31 } 

In [ ]:
#from sklearn.preprocessing import OrdinalEncoder

In [ ]:
#ordi_enco = OrdinalEncoder(categories = [Energinmering_des_order])

In [ ]:
#filt = (~df['Energimerking'].isna())
#ordi_enco.fit(df.loc[filt,['Energimerking']])

In [ ]:
#pd.Series(ordi_enco.transform(df.loc[filt,['Energimerking']])[:,0])

In [ ]:
df['Energimerking_encoded'] = df['Energimerking'].apply(lambda x: np.nan if pd.isnull(x) else Energimerking_enco[x])

In [ ]:
df['Energimerking_encoded']

In [ ]:
df['Eieform_bolig'].unique()

In [ ]:
df = pd.concat([df, pd.get_dummies(df['Eieform_bolig'])], axis = 1)

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
# predict the missing values in the Energimerking using KNN imputer

from sklearn.impute import KNNImputer

imput_knn = KNNImputer()
imputed__ =imput_knn.fit_transform(df.drop(columns = ['Address','Boligtype','Eieform_bolig','Byggeår','Energimerking','Beskrivelse', 'Eiendomsmegler']))
len(imputed__[:,-4])


In [ ]:
#imputed__[:,-4].shape

In [ ]:
s = pd.Series(imputed__[:,-4], index=df.index, name='encoded_energimerking_noNaN')

In [ ]:
df = pd.concat([df, s], axis = 1)

In [ ]:
df.shape

In [ ]:
# filling the the missing Energimerking in the column Energimerking_encoded with the predicted values in encoded_energimerking_noNaN

filt = (df['Energimerking_encoded'].isna())
df.loc[filt]

In [ ]:
# knn imputer can predict decimal values, because it looks into the 5 nearest neighbours and calculate the mean of the 
# Energimerking values for these rows and assign it to the missing values, thats way I round the values, to get a whole number 
#
df.loc[filt,'Energimerking_encoded'] = round(df.loc[filt,'encoded_energimerking_noNaN'])

In [ ]:
df['Energimerking_encoded'].unique()

In [ ]:
# now fill missing values in Energimerking

def get_key(x):
    for key, value in Energimerking_enco.items():
        if value == x:
            return key

df.loc[filt,'Energimerking'] = df.loc[filt, 'Energimerking_encoded'].apply(get_key)

In [ ]:
df['Energimerking'].isna().sum()

In [ ]:
del df['encoded_energimerking_noNaN']

In [ ]:
df.isna().sum()

In [ ]:
df.to_csv('data_cleaned_more.csv', index = False)

In [ ]:
data_inn_df = pd.read_csv('data_cleaned_more.csv')
data_inn_df.head()

In [ ]:
#df['Energimerking_simpl'] = df['Energimerking'].apply(lambda x: np.nan if pd.isnull(x) else Energimerking_simp(x))

In [ ]:
#plt.rcParams["figure.figsize"] = (12, 7)
#df['Energimerking'].value_counts().plot.bar()